In [11]:
import pickle
import numpy as np
import pandas as pd
import requests
import pprint
import datetime
import dateutil.parser


In [3]:
from youtube_api_crawler import crawler_api as crawler

In [4]:
### load random_forest
model = pickle.load(open('RandomForest_classifier.sav', 'rb'))
model


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [5]:
### load sentiment model
count_vectorizer = pickle.load(open('sentiment_countvectorizer.sav', 'rb'))
count_vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=8, min_df=1,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=frozenset({'its', 'same', 'bill', 'should', 'whose', 'when', 'sixty', 'very', 'under', 'becomes', 'forty', 'onto', 'twelve', 'something', 'though', 'every', 'whereas', 'put', 'across', 'some', 'which', 'one', 'yourself', 'what', 'how', 'toward', 'everything', 'beside', 'seem', 'into', 'an...'fill', 'often', 'thin', 'whoever', 'ie', 'a', 'was', 'someone', 'bottom', 'once', 'always', 'had'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [6]:
count_vectorizer.get_feature_names()

['audio',
 'music',
 'music video',
 'official',
 'official music',
 'official music video',
 'official video',
 'video']

In [8]:
crawler.search_list_by_keyword_http_request(search_word='Dua Lipa - Kiss and Make Up', 
                                            part = 'snippet', 
                                            region = 'US', 
                                            search_type = 'video')

{'kind': 'youtube#searchListResponse',
 'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/DhfGJD-wFqiF4qeZ9AEB2UG3p3I"',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'US',
 'pageInfo': {'totalResults': 1000000, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': '"XI7nbFXulYBIpL0ayR_gDh3eu1k/GE8sRWM116xl-vqEcV_CddYNI9E"',
   'id': {'kind': 'youtube#video', 'videoId': 'AX3Bsiq-13k'},
   'snippet': {'publishedAt': '2018-10-18T22:56:57.000Z',
    'channelId': 'UC-J-KZfRV8c13fOCkhXdLiQ',
    'title': 'Dua Lipa & BLACKPINK - Kiss and Make Up (Official Audio)',
    'description': 'Dua Lipa & BLACKPINK - Kiss and Make Up (Official Audio) The Complete Edition is here!! https://dualipa.co/completeedition Follow me online: ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/AX3Bsiq-13k/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/AX3Bsiq-13k/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {

In [16]:
yourdate = '2018-10-19T07:51:33.000Z'
yourdate

'2018-10-19T07:51:33.000Z'

In [17]:
now =datetime.datetime.now().isoformat()
now

'2018-12-19T12:10:07.658912'

In [19]:
def days_different(now, date):
    date = dateutil.parser.parse(date)
    now = dateutil.parser.parse(datetime.datetime.now().isoformat())
    a = datetime.date(now.year, now.month, now.day)
    b = datetime.date(date.year, date.month, date.day)
    return (a-b).days
days_different(now, yourdate)

61

In [146]:
def crawler_ml_modeled(search_query):
    responses = crawler.search_list_by_keyword_http_request(search_word = search_query, 
                                                            part ='snippet', 
                                                            region = 'US', 
                                                            search_type = 'video')
    #print(pprint.pprint(responses))
    #return responses
    if pd.isna(responses) == True:
        return responses
    else:
        for response in responses['items']:
            #return video_title
            video_title = response['snippet']['title']
            print(video_title)
            title_transform = count_vectorizer.transform([video_title]).toarray().reshape(-1)
            #print(title_transform)
            #return video_id
            video_id = response['id']['videoId']
            #return channelId
            channelId = response['snippet']['channelId']
            #return video_view
            video_view = int(crawler.search_list_by_id(video_id = video_id, part = 'statistics')['items'][0]['statistics']['viewCount'])
            
            log_video_view = np.log(video_view)
            #return channel_sub   
            channel_sub = int(crawler.search_list_by_channel(channelId = channelId, part = 'statistics')['items'][0]['statistics']['subscriberCount'])
            if channel_sub >= 100000:
                verified = 1
            else:
                verified = 0
                   
            # return available_time
            publish_date = response['snippet']['publishedAt']
            up_time = days_different(now, publish_date)
            features = np.concatenate((np.array((verified, up_time, log_video_view)),title_transform ),axis = None)
            #print(features)
            result = model.predict(features.reshape(1, -1))
            #print(result)
            if result == 1:
                return video_id
            else:
                continue 
    return np.nan

                

when the predicted results is 50/50, it random choice between 1 and 0, this can be improved by setting a threshold for 1, ie : 80% certainty

In [147]:
a = crawler_ml_modeled('Hugh Jackman, Keala Settle, Zac Efron, Zendaya & The Greatest Showman Ensemble	Hugh Jackman, Keala Settle, Zac Efron, Zendaya & The Greatest Showman Ensemble - The Greatest Show')

Hugh Jackman, Keala Settle, Zac Efron, Zendaya - The Greatest Show (Lyrics Video)
[0]
The Greatest Interview Ever! (Hugh Jackman, Zac Efron, Zendaya, Keala Settle) | The Greatest Showman
[1]


In [148]:
print(a)

-wd9sIDz46k


In [129]:
test_df = pd.read_excel('test_input.xlsx', header = 0)
test_df.head()

,artist,song_title
0,Desperate Man,Eric Church
1,Trip,Ella Mai
2,All I Want For Christmas Is You,Mariah Carey
3,Better,Khalid
4,Heaven,Kane Brown


In [99]:
test_df['search_keyword'] = test_df.song_title + ' - ' + test_df.artist

In [106]:
test_df['youtube_id'] = test_df.search_keyword.apply(lambda row: crawler_ml_modeled(row))

Eric Church - Desperate Man (Official Music Video)
Ella Mai - Trip (Audio)
Ella Mai - Trip
Mariah Carey - All I Want For Christmas Is You
Khalid - Better (Official Music Video)
Kane Brown - Heaven (Official Music Video)
Kelsea Ballerini - Miss Me More
Miss Me More - Kelsea Ballerini
Trans-Siberian Orchestra - Christmas Canon (Official Video)
Jingle Bell Rock / Bobby Helms 1957
Kane Brown - Lose It (Official Music Video)
Queen - Somebody To Love (Official Video)
lovelytheband - broken (official video)
Lady Gaga, Bradley Cooper - Shallow (A Star Is Born)
Loren Allred - Never Enough (Live Performance)
Hugh Jackman, Keala Settle, Zac Efron, Zendaya - The Greatest Show (Lyrics Video)
The Greatest Interview Ever! (Hugh Jackman, Zac Efron, Zendaya, Keala Settle) | The Greatest Showman
Khalid - Better (Official Music Video)
Chris Lane - I Don't Know About You
Rita Ora - Only Want You [Official Audio]
Rita Ora - Only Want You (Lyrics)
Rita Ora performs "Only Want You" Live in Germany
Rita Ora -

In [107]:
test_df.head()

,artist,song_title,search_keyword,youtube_id
0,Desperate Man,Eric Church,Eric Church - Desperate Man,sq93t3J4R5c
1,Trip,Ella Mai,Ella Mai - Trip,W16bk86xIY0
2,All I Want For Christmas Is You,Mariah Carey,Mariah Carey - All I Want For Christmas Is You,yXQViqx6GMY
3,Better,Khalid,Khalid - Better,x3bfa3DZ8JM
4,Heaven,Kane Brown,Kane Brown - Heaven,dRX0wDNK6S4


In [113]:
test_df['youtube_link'] = 'https://www.youtube.com/watch?v=' + test_df.youtube_id
test_df.head()

,artist,song_title,search_keyword,youtube_id,youtube_link
0,Desperate Man,Eric Church,Eric Church - Desperate Man,sq93t3J4R5c,https://www.youtube.com/watch?v=sq93t3J4R5c
1,Trip,Ella Mai,Ella Mai - Trip,W16bk86xIY0,https://www.youtube.com/watch?v=W16bk86xIY0
2,All I Want For Christmas Is You,Mariah Carey,Mariah Carey - All I Want For Christmas Is You,yXQViqx6GMY,https://www.youtube.com/watch?v=yXQViqx6GMY
3,Better,Khalid,Khalid - Better,x3bfa3DZ8JM,https://www.youtube.com/watch?v=x3bfa3DZ8JM
4,Heaven,Kane Brown,Kane Brown - Heaven,dRX0wDNK6S4,https://www.youtube.com/watch?v=dRX0wDNK6S4


In [114]:
test_df.to_csv('test_output.csv', index = False)